In [1]:
from torchtext import data
import torch
import numpy as np
from torch import nn
import torch.optim as optim
import copy
import torch.nn.functional as F
import subprocess
import socket
from torchtext.data import ReversibleField, BucketIterator

from torch.nn.utils.rnn import pack_padded_sequence as pack
from torch.nn.utils.rnn import pad_packed_sequence as unpack
path = '/media/ubuntu/1TO/DTU/courses/DeepLearning/DeepLearning_summarization/SampleData/train_short_unique_countries.csv'
path_val = '/media/ubuntu/1TO/DTU/courses/DeepLearning/DeepLearning_summarization/SampleData/validation_short_unique_countries.csv'
BATCH_SIZE = 16
device = torch.device("cpu")

In [2]:
TEXT = data.Field(init_token='<bos>', eos_token='<eos>', sequential=True, include_lengths=True)
#    LABEL = data.Field(init_token='<bos>', eos_token='<eos>', sequential=True)
train_set = data.TabularDataset(path, 'CSV', fields=[('data', TEXT), ('label', TEXT)], skip_header=True ) 
validation_set = data.TabularDataset(path_val, 'CSV', fields=[('data', TEXT), ('label', TEXT)], skip_header=True ) 

TEXT.build_vocab(train_set)#, vectors="glove.6B."+str(glove_dim)+"d")
#    LABEL.vocab = TEXT.vocab
vocab = copy.deepcopy(TEXT.vocab)


#This is the glove embedding for every word in vocab, we dont need to load it into memory
#w = embed.weight.data.copy_(vocab.vectors)

#Data loader iterator
dataset_iter = data.Iterator(train_set, batch_size=BATCH_SIZE, device=device,
        train=True, shuffle=False, repeat=False, sort=True, sort_key=lambda x: len(x.data))
dataset_iter_train_val = data.Iterator(train_set, batch_size=1, device=device,
        train=True, shuffle=False, repeat=False, sort=False)

dataset_iter_val = data.Iterator(validation_set, batch_size=1, device=device,
        train=True, shuffle=True, repeat=False, sort=False)

In [3]:
def display(x):
    s = ' '.join([vocab.itos[i] for i in x if vocab.itos[i] not in ['<bos>', '<pad>']])
    return s.split('<eos>')[0]

In [4]:
for batchData in dataset_iter:
    print(batchData.data[1])
    print(batchData.data[0].size())
    print('Data')
    print(display(batchData.data[0][:,0]))
    print('Labels')
    print(batchData.label[1])
    print(batchData.label[0].size())
    print(display(batchData.label[0][:,0]))
    
    break
    

tensor([7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7])
torch.Size([7, 16])
Data
14 your sleep 15 nature 
Labels
tensor([4, 5, 5, 6, 6, 7, 4, 6, 4, 5, 6, 4, 6, 5, 5, 5])
torch.Size([7, 16])
14 15 


In [24]:
class PointerGenerator(nn.Module):
    def __init__(self):
        super(PointerGenerator, self).__init__()
        self.embedding = nn.Embedding(44, 20)
        self.encoder = nn.LSTM(input_size=20, hidden_size=100)
        self.decoder = nn.LSTM(input_size=20, hidden_size=100)
        self.out = nn.Linear(in_features=100, out_features=44)
        self.attention = nn.Linear(in_features=100+100, out_features=50, bias=True)
        self.att_out = nn.Linear(in_features=50, out_features=1, bias=False)
        self.linear_in = nn.Linear(in_features=100, out_features=100, bias=True)
        self.linear_vocab1 = nn.Linear(in_features = 100+100, out_features=50, bias=True)
        self.linear_vocab2 = nn.Linear(in_features=50, out_features=44, bias=False)
        self.softmax = nn.Softmax(dim=-1)
        self.linear_pgen = nn.Linear(in_features=100+100+20, out_features=1, bias=True)
        self.sigmoid = nn.Sigmoid()
    def forward(self,y, x=None,  hid_enc=None, cell_enc=None, x_length=None, out_enc=None, real_x=None):
        batch_size = y.size()[1]
        big_vocab_size = torch.max(real_x).item()
        if not x is None:
            x_emb = self.embedding(x)
            if x_length is not None:
                # Lengths data is wrapped inside a Tensor.
                lengths_list = x_length.view(-1).tolist()
                packed_emb = pack(x_emb, lengths_list)

            out_enc, (hid_enc, cell_enc) = self.encoder(packed_emb)
            hid_enc = hid_enc
            cell_enc = cell_enc
            if x_length is not None:
                out_enc = unpack(out_enc)[0]
        
        y_emb = self.embedding(y)
        
        out_enc_ = out_enc
        out_dec, (hid_dec, cell_dec) = self.decoder(y_emb, (hid_enc, cell_enc))
        
        tgt_len, batch_size, tgt_dim = out_dec.size()
        src_len = out_enc.size()[0]
        out_dec_ = out_dec.view(batch_size * tgt_len, tgt_dim)
        out_dec_ = self.linear_in(out_dec_)
        out_dec_ = out_dec_.view(batch_size, tgt_len, tgt_dim)
        out_enc = out_enc.permute([1,2,0])
        # (batch, t_len, d) x (batch, d, s_len) --> (batch, t_len, s_len)
        align = torch.bmm(out_dec_, out_enc)
        mask = sequence_mask(x_length, max_len=align.size(-1))
        mask = mask.unsqueeze(1)  # Make it broadcastable.

        align.masked_fill_(1 - mask, -float('inf'))
        
        align_vectors = F.softmax(align.view(batch_size*tgt_len, src_len), -1)
        align_vectors = align_vectors.view(batch_size, tgt_len, src_len)
        out_enc = out_enc.permute([0,2,1])
        c = torch.bmm(align_vectors, out_enc)
    
        in_pvocab = torch.cat((out_dec, c.permute([1,0,2])), dim=2).view(-1, 200)
        pvocab = self.linear_vocab2(self.linear_vocab1(in_pvocab)).view(-1, batch_size, 44)
        p_vocab = self.softmax(pvocab)
        
        in_pgen = torch.cat((out_dec, c.permute([1,0,2]), y_emb), dim=2).view(-1, 220)
        pgen = self.sigmoid(self.linear_pgen(in_pgen)).view(-1, batch_size,1)
        p_vocab = pgen * p_vocab
        if big_vocab_size > 44:
            p_vocab = torch.cat((p_vocab, torch.zeros(tgt_len,batch_size,big_vocab_size-44)), dim=2)
        
                            
        return p_vocab, hid_dec, cell_dec, out_enc_
    
model = PointerGenerator()

#criterion = nn.CrossEntropyLoss(reduction='none')
criterion = nn.NLLLoss(reduction='none')
optimizer = optim.Adam(model.parameters(), lr=0.001)        
        

In [27]:
def acc(y, target):
    return torch.mean((y==target).float())
def label_mask(y):
    """y is the label tensor (no glove embedding)
       returns the mask to use for negating the padding"""
    mask = (y != vocab.stoi['<pad>'])
    return mask.float()
def compute_loss(y, t, mask=True):
    """
    predictions = y[:-1].permute([1,2,0])
    target = t[1:].permute([1,0])
    loss = criterion(predictions, target)"""
    batch_size = y.size()[1]
    predictions = y[:-1].contiguous()
    predictions=predictions.view(-1,predictions.size()[2])

    target = t[1:].view(-1)
    loss = criterion(torch.log(predictions), target).view(-1,batch_size)
    if mask:
        y_mask = label_mask(t[1:])
        loss_mask = torch.sum(y_mask * loss, dim=0)
        mean_loss = loss_mask/torch.sum(y_mask,dim=0)
        final_loss = torch.mean(mean_loss)
        return final_loss
    else:
        return torch.mean(loss)
def sequence_mask(lengths, max_len=None):
    """
    Creates a boolean mask from sequence lengths.
    """
    batch_size = lengths.numel()
    max_len = max_len or lengths.max()
    return (torch.arange(0, max_len)
            .type_as(lengths)
            .repeat(batch_size, 1)
            .lt(lengths.unsqueeze(1)))
def filter_unk(x, vocab_size):
    return x * (x <= vocab_size+3).long()

In [28]:
for k in range(10):
    model.train()
    mean_losses = []
    accs = []
    for batchData in dataset_iter:
        real_x = batchData.data[0]
        x = filter_unk(real_x, 40)
        real_y = batchData.label[0]
        y = filter_unk(real_y, 40)
        out, _, _ ,_= model(y, x, x_length=batchData.data[1], real_x=real_x)
        y_hat = torch.argmax(out,-1)
        accuracy = acc(y_hat[:-1], y[1:])
        
        final_loss = compute_loss(out, real_y)
        optimizer.zero_grad()
        final_loss.backward()
        optimizer.step()
        mean_losses.append(final_loss)
        accs.append(accuracy)
    print('EPOCH ', k+1, ' Train: mean loss: ', torch.mean(torch.stack(mean_losses)).item(), 'mean acc: ', torch.mean(torch.stack(accs)).item())
mean_losses = []
accs = []    
model.eval()
for batchData in dataset_iter_val:
    out, _, _ ,_= model(batchData.label[0], batchData.data[0], x_length=batchData.data[1], real_x=real_x)
    y_hat = torch.argmax(out,-1)
    accuracy = acc(y_hat[:-1], batchData.label[0][1:])
    loss = compute_loss(out, batchData.label[0])
    mean_loss = torch.mean(loss)
    mean_losses.append(mean_loss)
    accs.append(accuracy)
print('Validation TF: mean loss: ', torch.mean(torch.stack(mean_losses)).item(), 'mean acc: ', torch.mean(torch.stack(accs)).item())
accs=[]
lines_y = []
lines_t = []
for batchData in dataset_iter_val:
    x =  batchData.data[0]
    y = batchData.label[0]
    out, hid_dec , cell_dec, out_enc = model(y[0:1,:], x=x, x_length=batchData.data[1])
    w_hat = torch.argmax(out,-1)
    out_sequence = [w_hat]
    for i in range(21):
        out, hid_dec, cell_dec,_ = model(w_hat, x=None, hid_enc=hid_dec, cell_enc=cell_dec, x_length=batchData.data[1], out_enc=out_enc)
        w_hat = torch.argmax(out,-1)
        out_sequence.append(w_hat)
    y_hat = torch.stack(out_sequence).squeeze()
    y_true = batchData.label[0][1:].squeeze()
    accuracy = acc(y_hat[:y_true.size()[0]], y_true)
    accs.append(accuracy)
    lines_y.append(display(y_hat))
    lines_t.append(display(y_true))
line_y = '\n'.join(lines_y)
line_t = '\n'.join(lines_t) 
y_file = open('output', 'w')
y_file.write(line_y)
y_file.close()
t_file = open('reference', 'w')
t_file.write(line_t)
t_file.close()
bleu_score = str(subprocess.Popen("perl multi-bleu.perl reference < output", shell=True, stdout=subprocess.PIPE).stdout.read()).split(' ')[2][:-1]
print('BLEU score:', bleu_score)
print('Validation non TF: mean acc: ', torch.mean(torch.stack(accs)).item())


RuntimeError: Target 63 out of bounds at /opt/conda/conda-bld/pytorch_1533739672741/work/aten/src/THNN/generic/ClassNLLCriterion.c:56

In [17]:
torch.max(real_x).item()


51

In [ ]:
len(dataset_iter_val)

In [ ]:
label_mask(y)